In [ ]:
import json
import sys
import numpy as np
import scipy.sparse as ssparse
import scipy.io as sio
import multiprocessing
from multiprocessing import Pool 
import time 
import os
import datetime
import warnings
import pandas as pd 
from ctypes import *
import gc 
from heapq import nlargest
import shapely.wkt

pd.set_option('display.max_columns', 10)
sys.setrecursionlimit(2000)

absolute_path = '.'
sys.path.insert(0, absolute_path+'/../')
sys.path.insert(0, '/home/bingyu/')
from sp import interface 


In [12]:
project_folder='los_angeles'

### Read in the edge attribute for volume delay calculation later
edges_df = pd.read_csv(absolute_path+'/projects/{}/network_inputs/osm_edges_scag.csv'.format(project_folder))
nodes_df = pd.read_csv(absolute_path+'/projects/{}/network_inputs/osm_nodes_scag.csv'.format(project_folder))

### results df
### ATTENTION: consider which result file to use
results_df = pd.read_csv(absolute_path+'/projects/{}/simulation_outputs/edges_df/edges_df_scenscag_5pct_bc30_r0_DYna_HR0_QT1.csv'.format(project_folder))
results_df = results_df.replace([np.inf, -np.inf], np.nan)
edges_df = edges_df.merge(results_df[['edge_id_igraph', 't_avg']], how='left', on='edge_id_igraph')
edges_df['fft'] = np.where(np.isnan(edges_df['t_avg']), edges_df['fft'], edges_df['t_avg'])
edges_df = edges_df.drop(columns=['t_avg'])

## damage df
damage_df0 = pd.read_csv(absolute_path+'/projects/{}/network_inputs/bridge_closure/bridge_closure_day90.csv'.format(project_folder))
damage_links = damage_df0['OSMWayID1'].values.tolist() + damage_df0.dropna(subset=['OSMWayID2'])['OSMWayID2'].values.tolist()
damage_df = pd.DataFrame({'edge_osmid': damage_links})
damage_df = damage_df.merge(edges_df[['edge_osmid', 'edge_id_igraph']], how='left', on='edge_osmid')
edges_df.loc[edges_df['edge_id_igraph'].isin(damage_df['edge_id_igraph']), 'fft'] = 10e7

node_count = max(len(np.unique(edges_df['start_igraph'])), len(np.unique(edges_df['end_igraph'])))
g_coo = ssparse.coo_matrix((edges_df['fft']*1.2, (edges_df['start_igraph'], edges_df['end_igraph'])), shape=(node_count, node_count))
edges_df = edges_df[['edge_id_igraph', 'start_sp', 'end_sp', 'length', 'capacity', 'fft', 'type']]
sio.mmwrite(absolute_path+'/projects/{}/simulation_outputs/network_sparse_port_traffic.mtx'.format(project_folder), g_coo)


In [16]:
### read in the new graph for computing travel time of port traffic
g = interface.readgraph(bytes(absolute_path+'/projects/{}/simulation_outputs/network_sparse_port_traffic.mtx'.format(project_folder), encoding='utf-8'))

### port traffic od
port_ods = pd.read_csv(absolute_path+'/projects/{}/demand_inputs/port_to_outside_zone_od.csv'.format(project_folder))
port_ods = pd.merge(port_ods, nodes_df[['node_osmid', 'node_id_igraph']], how='left', left_on='port_origin_osmid', right_on='node_osmid')
port_ods = pd.merge(port_ods, nodes_df[['node_osmid', 'node_id_igraph']], how='left', left_on='outside_destin_osmid', right_on='node_osmid', suffixes=['_O', '_D'])
port_ods['origin_sp'] = port_ods['node_id_igraph_O'] + 1 ### the node id in module sp is 1 higher than igraph id
port_ods['destin_sp'] = port_ods['node_id_igraph_D'] + 1

for od in port_ods.itertuples():
    print('origin osmid ', getattr(od, 'port_origin_osmid'))
    print('destin osmid ', getattr(od, 'outside_destin_osmid'))
    origin_ID = getattr(od, 'origin_sp')
    destin_ID = getattr(od, 'destin_sp')
    od_straight_line_dist = getattr(od, 'straight_line_dist')
    print('od straigtline distance (m) is ', od_straight_line_dist)

    sp = g.dijkstra(origin_ID, destin_ID) 
    sp_dist = sp.distance(destin_ID) 
    print('travel time (s) is ', sp_dist)
    break

    ### the rest should follow the main code

origin osmid  123156182
destin osmid  87739120
od straigtline distance (m) is  261534.8961740201
travel time (s) is  20211.624835573428
